# MIMIC3c aggregated data
https://www.kaggle.com/datasets/drscarlat/mimic3c

In [1]:
import sys
path_append = "../../"
sys.path.append(path_append)  # Go up one directory from where you are.

import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd

df = pd.read_csv(path_append + "../data/mimic_custom/mimic3c.csv")

In [3]:
df['religion'].fillna('NOT SPECIFIED', inplace=True)
df['marital_status'].fillna('UNKNOWN (DEFAULT)', inplace=True)
df.dropna(inplace=True, axis=0)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58951 entries, 0 to 58975
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   hadm_id           58951 non-null  int64  
 1   gender            58951 non-null  object 
 2   age               58951 non-null  int64  
 3   LOSdays           58951 non-null  float64
 4   admit_type        58951 non-null  object 
 5   admit_location    58951 non-null  object 
 6   AdmitDiagnosis    58951 non-null  object 
 7   insurance         58951 non-null  object 
 8   religion          58951 non-null  object 
 9   marital_status    58951 non-null  object 
 10  ethnicity         58951 non-null  object 
 11  NumCallouts       58951 non-null  float64
 12  NumDiagnosis      58951 non-null  float64
 13  NumProcs          58951 non-null  float64
 14  AdmitProcedure    58951 non-null  object 
 15  NumCPTevents      58951 non-null  float64
 16  NumInput          58951 non-null  float64
 17

In [5]:
df_selected_target = df[['LOSdays']]
df = df.drop('LOSdays', axis=1)

In [6]:
# from tools.preprocessing.data_frame import auto_preprocess_dataframe
# target_columns = ['ExpiredHospital']
# drop_columns = ['hadm_id', 'AdmitDiagnosis', 'LOSgroupNum', 'AdmitProcedure']
# encode_columns = ['gender', 'admit_type', 'admit_location', 'insurance', 'religion', 'marital_status',]
# df, description = auto_preprocess_dataframe(df, target_columns=target_columns, drop_columns=drop_columns, encode_columns=encode_columns) 

In [7]:
# import torch
# from sklearn.model_selection import train_test_split
# from tools.preprocessing.template_dataset import TemplateDataset

# train_df, test_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)
# # predict the next value in the sequence
# train_df_x = train_df.iloc[:, :-1] # all columns except the last one
# train_df_y = train_df.iloc[:, -1:] # only the last column

# test_df_x = test_df.iloc[:, :-1] # all columns except the last one
# test_df_y = test_df.iloc[:, -1:] # only the last column

# print('train df shape: ', train_df.shape)
# print('test df shape: ', test_df.shape)
# trainset = TemplateDataset(train_df_x, train_df_y)
# testset = TemplateDataset(test_df_x, test_df_y)

In [8]:
# from tools.setting.data_config import DataConfig
# from tools.setting.ml_params import MLParameters
# from trainer_hub import TrainerHub

# num_features = description['num_features']
# num_classes = description['num_classes']
# data_config = DataConfig(dataset_name = 'mimic3', task_type='binary_classification', obs_shape=[num_features], label_size=num_classes)

# #  Set training configuration from the AlgorithmConfig class, returning them as a Namespace object.
# ml_params = MLParameters(ccnet_network = 'tabnet', encoder_network = 'none')
# ml_params.algorithm.error_function = 'mae'
# ml_params.model.ccnet_config.num_layers = 3
# ml_params.training.num_epoch = 5

# # Set the device to GPU if available, else CPU
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# # Initialize the TrainerHub class with the training configuration, data configuration, device, and use_print and use_wandb flags
# trainer_hub = TrainerHub(ml_params, data_config, device, use_print=True, use_wandb= False) 

In [9]:
# trainer_hub.train(trainset, testset)

In [10]:
from nn.tabnet import TabNet 
from tools.setting.ml_params import ModelConfig
import torch

class AttributeClassifier(torch.nn.Module):
    def __init__(self, input_size, output_size, num_layers=3, hidden_size=256):
        super(AttributeClassifier, self).__init__()
        
        model_config = ModelConfig('tabnet')
        model_config.num_layers = num_layers
        model_config.d_model = hidden_size
        
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        
        # Create a list to hold all layers
        layers = []
        
        # Input layer
        layers.append(torch.nn.Linear(input_size, hidden_size))
        layers.append(torch.nn.ReLU())
        
        ## Add TabNet layers
        layers.append(TabNet(model_config))
        layers.append(torch.nn.ReLU())

        # Output layer
        layers.append(torch.nn.Linear(hidden_size, output_size))
        
        # Register all layers
        self.layers = torch.nn.Sequential(*layers)

    def forward(self, x):
        x = self.layers(x)
        return torch.sigmoid(x)

In [11]:
import tqdm
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler, LabelEncoder


# data = pd.read_csv(path_append + "../data/mimic_custom/mimic3c.csv")
df.drop(columns=['hadm_id', 'AdmitDiagnosis', 'LOSgroupNum', 'AdmitProcedure'], inplace=True)


y = LabelEncoder().fit_transform(df['ExpiredHospital'])
X = df.drop('ExpiredHospital', axis=1)

for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = LabelEncoder().fit_transform(X[col])
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

dataset = TensorDataset(X_tensor, y_tensor)

dataset_loader = DataLoader(dataset, batch_size=64, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AttributeClassifier(input_size=X.shape[1], output_size=len(set(y)), num_layers=3, hidden_size=256)
model.to(device)

# Loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# 데이터 세트 분할
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 훈련 및 검증 데이터셋 생성
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

def evaluate_model(model, data_loader):
    model.eval()  # 모델을 평가 모드로 설정
    true_labels, predicted_labels = [], []
    
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(predicted.cpu().numpy())

    accuracy = accuracy_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels, average='weighted')
    return accuracy, f1

def train_model(train_loader, val_loader):
    model.train()
    for epoch in tqdm.tqdm(range(10)):
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, torch.nn.functional.one_hot(labels, num_classes=model.output_size).float())

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # 에폭마다 성능 평가
        train_accuracy, train_f1 = evaluate_model(model, train_loader)
        val_accuracy, val_f1 = evaluate_model(model, val_loader)
        print(f'Epoch [{epoch + 1}/10], Loss: {loss.item():.4f}, Train Acc: {train_accuracy:.4f}, Train F1: {train_f1:.4f}, Val Acc: {val_accuracy:.4f}, Val F1: {val_f1:.4f}')

# 모델 훈련 시작
train_model(train_loader, val_loader)

 10%|█         | 1/10 [00:18<02:49, 18.78s/it]

Epoch [1/10], Loss: 0.3920, Train Acc: 0.9337, Train F1: 0.9269, Val Acc: 0.9316, Val F1: 0.9249


 20%|██        | 2/10 [00:37<02:27, 18.46s/it]

Epoch [2/10], Loss: 0.3838, Train Acc: 0.9348, Train F1: 0.9258, Val Acc: 0.9317, Val F1: 0.9223


 30%|███       | 3/10 [00:55<02:08, 18.32s/it]

...

Epoch [8/10], Loss: 0.3377, Train Acc: 0.9406, Train F1: 0.9357, Val Acc: 0.9329, Val F1: 0.9275


 90%|█████████ | 9/10 [02:42<00:17, 17.83s/it]

Epoch [9/10], Loss: 0.3849, Train Acc: 0.9423, Train F1: 0.9343, Val Acc: 0.9358, Val F1: 0.9269


100%|██████████| 10/10 [03:00<00:00, 18.03s/it]

Epoch [10/10], Loss: 0.3847, Train Acc: 0.9439, Train F1: 0.9384, Val Acc: 0.9348, Val F1: 0.9282
